<a href="https://colab.research.google.com/github/gabrielraya/vanilla-diffusion-model/blob/main/Vanilla_Diffusion_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vanilla Diffusion models 

This notebook provides the minimal code required for you to experience how diffusion models work. The implementation 

In [6]:
!git clone https://github.com/gabrielraya/vanilla-diffusion-model.git

Cloning into 'vanilla-diffusion-model'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 5 (delta 1), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [7]:
import os 


# Create directories for experiemental logs
workdir = "./"
sample_dir = os.path.join(workdir, "samples")
os.makedirs(sample_dir, exist_ok=True)

In [17]:
# @title Training configuration 

batch_size = 64 #@param {"type":"integer"}
n_iters = 1000 #@param {"type":"integer"}
snapshot_freq = 50000 #@param {"type":"integer"}
grid_size = 25 #@param {"type":"integer"}
dataset = 'CIFAR10' #@param ['MNIST', 'CIFAR10', 'CIFAR10'] {"type": "string"}


class Config():
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)


training = Config(batch_size=batch_size, n_iters=n_iters)
sampling = Config(grid_size=grid_size)
config   = Config(training=training, sampling=sampling)

In [8]:
from vanilla-diffusion-model import datasets


# Build data iterators
train_ds, eval_ds, _ = datasets.get_dataset(config,
                                            uniform_dequantization=config.data.uniform_dequantization)
train_iter = iter(train_ds)  # pytype: disable=wrong-arg-types
eval_iter = iter(eval_ds)  # pytype: disable=wrong-arg-types
# Create data normalizer and its inverse
scaler = datasets.get_data_scaler(config)
inverse_scaler = datasets.get_data_inverse_scaler(config)

SyntaxError: ignored

In [3]:
jax.device_count()

1

In [18]:
config.training.batch_size

64

In [2]:
# @title Load the score-based model
sde = 'VESDE' #@param ['VESDE', 'VPSDE', 'subVPSDE'] {"type": "string"}
if sde.lower() == 'vesde':
  from configs.ve import cifar10_ncsnpp_continuous as configs
  ckpt_filename = "exp/ve/cifar10_ncsnpp_continuous/checkpoint_24"
  config = configs.get_config()  
  sde = VESDE(sigma_min=config.model.sigma_min, sigma_max=config.model.sigma_max, N=config.model.num_scales)
  sampling_eps = 1e-5
elif sde.lower() == 'vpsde':
  from configs.vp import cifar10_ddpmpp_continuous as configs
  ckpt_filename = "exp/vp/cifar10_ddpmpp_continuous/checkpoint_8"  
  config = configs.get_config()
  sde = VPSDE(beta_min=config.model.beta_min, beta_max=config.model.beta_max, N=config.model.num_scales)
  sampling_eps = 1e-3
elif sde.lower() == 'subvpsde':
  from configs.subvp import cifar10_ddpmpp_continuous as configs
  ckpt_filename = "exp/subvp/cifar10_ddpmpp_continuous/checkpoint_26"
  config = configs.get_config()
  sde = subVPSDE(beta_min=config.model.beta_min, beta_max=config.model.beta_max, N=config.model.num_scales)
  sampling_eps = 1e-3

batch_size =   64#@param {"type":"integer"}
local_batch_size = batch_size // jax.local_device_count()
config.training.batch_size = batch_size
config.eval.batch_size = batch_size

random_seed = 0 #@param {"type": "integer"}
rng = jax.random.PRNGKey(random_seed)
rng, run_rng = jax.random.split(rng)
rng, model_rng = jax.random.split(rng)
score_model, init_model_state, initial_params = mutils.init_model(run_rng, config)
optimizer = losses_lib.get_optimizer(config).create(initial_params)

state = mutils.State(step=0, optimizer=optimizer, lr=config.optim.lr,
                      model_state=init_model_state,
                      ema_rate=config.model.ema_rate,
                      params_ema=initial_params,
                      rng=rng)  # pytype: disable=wrong-keyword-args
sigmas = mutils.get_sigmas(config)
scaler = datasets.get_data_scaler(config)
inverse_scaler = datasets.get_data_inverse_scaler(config)
state = utils.load_training_state(ckpt_filename, state)

ModuleNotFoundError: ignored